In [1]:
model_ckp = "bert-base-uncased"
batch_size = 32

In [2]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 12.5 MB/s eta 0:00:00


In [3]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.9 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=dbcbc3ec34cc5adab9fc1d1b90b5ac1f495cc4cf6e61cbd051efa1f637167ff4
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools


In [4]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
from datasets import load_dataset

dataset = load_dataset("minoosh/IEMOCAP_Text")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating Session1 split:   0%|          | 0/1085 [00:00<?, ? examples/s]

Generating Session3 split:   0%|          | 0/1151 [00:00<?, ? examples/s]

Generating Session5 split:   0%|          | 0/1241 [00:00<?, ? examples/s]

Generating Session2 split:   0%|          | 0/1023 [00:00<?, ? examples/s]

Generating Session4 split:   0%|          | 0/1031 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/minoosh___parquet/minoosh--IEMOCAP_Text-8b3962ea9075930e/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    Session1: Dataset({
        features: ['emotion', 'transcription'],
        num_rows: 1085
    })
    Session3: Dataset({
        features: ['emotion', 'transcription'],
        num_rows: 1151
    })
    Session5: Dataset({
        features: ['emotion', 'transcription'],
        num_rows: 1241
    })
    Session2: Dataset({
        features: ['emotion', 'transcription'],
        num_rows: 1023
    })
    Session4: Dataset({
        features: ['emotion', 'transcription'],
        num_rows: 1031
    })
})

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_ckp)

In [9]:
def preprocess_function(examples):
    return tokenizer(examples["transcription"], truncation=True)

In [10]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1085 [00:00<?, ? examples/s]

Map:   0%|          | 0/1151 [00:00<?, ? examples/s]

Map:   0%|          | 0/1241 [00:00<?, ? examples/s]

Map:   0%|          | 0/1023 [00:00<?, ? examples/s]

Map:   0%|          | 0/1031 [00:00<?, ? examples/s]

In [11]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
import evaluate

accuracy = evaluate.load("accuracy")

In [13]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [14]:
dataset = dataset.rename_column("emotion", "label")

In [15]:
labels = dataset["Session1"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label["1"]

'hap'

In [16]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    model_ckp, num_labels=4, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [17]:
tokenized_dataset = tokenized_dataset.rename_column("emotion", "label")

In [18]:
import torch
train_dataset = torch.utils.data.ConcatDataset([tokenized_dataset['Session1'], tokenized_dataset['Session2'], tokenized_dataset['Session3']])

In [19]:
train_dataset

In [20]:
'''splited_tokenized_dataset = tokenized_dataset['train'].train_test_split(test_size=0.2)
splited_tokenized_dataset'''

"splited_tokenized_dataset = tokenized_dataset['train'].train_test_split(test_size=0.2)\nsplited_tokenized_dataset"

In [21]:
training_args = TrainingArguments(
    output_dir="finetuned_bert-base-uncased",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_steps=10,
    num_train_epochs=20,
    weight_decay=0.01,
    warmup_ratio=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    report_to='wandb',
    run_name='finetune_bert_base_on_IEMOCAP_text'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=tokenized_dataset["Session4"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/minoosh/finetuned_bert-base-uncased into local empty directory.


Download file pytorch_model.bin:   0%|          | 24.0k/418M [00:00<?, ?B/s]

Download file runs/Feb08_10-43-45_18b2d2006935/events.out.tfevents.1675853140.18b2d2006935.276.0:  45%|####5  …

Download file runs/Feb08_12-04-49_f873344109b0/1675857989.3221283/events.out.tfevents.1675857989.f873344109b0.…

Download file runs/Feb08_10-12-51_f339a4904974/1675851181.5398304/events.out.tfevents.1675851181.f339a4904974.…

...

Clean file runs/Feb25_10-04-17_13449c46355a/1677319478.418264/events.out.tfevents.1677319478.13449c46355a.897.…

Clean file runs/Feb25_10-04-17_13449c46355a/events.out.tfevents.1677319478.13449c46355a.897.0:  22%|##2       …

Clean file training_args.bin:  29%|##8       | 1.00k/3.50k [00:00<?, ?B/s]

Clean file runs/Feb08_14-15-00_144a3875a8da/1675865810.3426144/events.out.tfevents.1675865810.144a3875a8da.224…

Clean file pytorch_model.bin:   0%|          | 1.00k/418M [00:00<?, ?B/s]

In [22]:
import torch
torch.cuda.empty_cache()

In [23]:
results = trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3259
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2040
  Number of trainable parameters = 109485316
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: minooshayan97 (minoosh). Use `wandb login --relogin` to force relogin


The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: transcription. If transcription are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.276100,1.322480,0.337536
2,0.984700,1.079249,0.550921
3,0.688200,0.925969,0.638215
4,0.509900,0.907158,0.663434
5,0.461400,0.911465,0.686712
6,0.340600,1.002211,0.675073
7,0.189000,1.088107,0.675073
8,0.217900,1.152015,0.671193
9,0.208500,1.256748,0.689622
10,0.191400,1.207387,0.682832


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: transcription. If transcription are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1031
  Batch size = 32
Saving model checkpoint to finetuned_bert-base-uncased/checkpoint-102
Configuration saved in finetuned_bert-base-uncased/checkpoint-102/config.json
Model weights saved in finetuned_bert-base-uncased/checkpoint-102/pytorch_model.bin
tokenizer config file saved in finetuned_bert-base-uncased/checkpoint-102/tokenizer_config.json
Special tokens file saved in finetuned_bert-base-uncased/checkpoint-102/special_tokens_map.json
tokenizer config file saved in finetuned_bert-base-uncased/tokenizer_config.json
Special tokens file saved in finetuned_bert-base-uncased/special_tokens_map.json
The following columns in the evaluation set don't have a correspond

In [24]:
trainer.evaluate(tokenized_dataset['Session5'])

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: transcription. If transcription are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1241
  Batch size = 32


{'eval_loss': 1.1947249174118042,
 'eval_accuracy': 0.6792908944399678,
 'eval_runtime': 4.2839,
 'eval_samples_per_second': 289.688,
 'eval_steps_per_second': 9.104,
 'epoch': 20.0}

In [25]:
trainer.push_to_hub()

Saving model checkpoint to finetuned_bert-base-uncased
Configuration saved in finetuned_bert-base-uncased/config.json
Model weights saved in finetuned_bert-base-uncased/pytorch_model.bin
tokenizer config file saved in finetuned_bert-base-uncased/tokenizer_config.json
Special tokens file saved in finetuned_bert-base-uncased/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.
remote: Scanning LFS files of refs/heads/main for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/minoosh/finetuned_bert-base-uncased
   9753ce7..f3af2c1  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/minoosh/finetuned_bert-base-uncased
   9753ce7..f3af2c1  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.679290894439

'https://huggingface.co/minoosh/finetuned_bert-base-uncased/commit/f3af2c18d2df97056e90c96cb166c5ab51094376'